<a href="https://colab.research.google.com/github/onemoremoon/random/blob/main/BacktestTemplate2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Install Required Libraries and Import Libraries
# This section installs all necessary Python packages for the backtesting script
!pip install yfinance pandas numpy matplotlib vectorbt -q
import yfinance as yf
import pandas as pd
import numpy as np
import vectorbt as vbt
import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.0 MB/s eta 0:00:00


In [2]:
#@title Function to download and clean financial data from Yahoo Finance
def get_clean_financial_data(ticker, start_date, end_date):
    """Downloads and prepares stock data from Yahoo Finance."""
    print(f"\nDownloading data for {ticker} from {start_date} to {end_date}...")
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    if data.empty:
        print(f"Error: No data found for ticker '{ticker}'. Please check the ticker symbol and date range.")
        return None
    data.columns = data.columns.get_level_values(0) if isinstance(data.columns, pd.MultiIndex) else data.columns
    data = data.ffill()
    if data.index.tz is not None:
        data.index = data.index.tz_localize(None)
    print(f"Data download complete. Shape: {data.shape}")
    return data

In [3]:
#@title Function to format raw backtest results into readable trade logs
def format_trade_logs(pf, trade_logs):
    """
    Formats the raw vectorbt trade logs into a more readable format.
    - Removed Gross PnL column
    - Shows Net PnL after fees
    - Includes accurate portfolio values
    """
    if trade_logs.empty:
        return pd.DataFrame()

    formatted_logs = trade_logs.copy()
    formatted_logs['Exit Timestamp'] = pd.to_datetime(formatted_logs['Exit Timestamp'])

    portfolio_values = pf.value()
    formatted_logs['Portfolio Value'] = portfolio_values.reindex(
        formatted_logs['Exit Timestamp'],
        method='ffill'
    ).values

    formatted_logs['Entry Fees'] = formatted_logs['Entry Fees'].abs()
    formatted_logs['Exit Fees'] = formatted_logs['Exit Fees'].abs()
    formatted_logs['Total Fees'] = formatted_logs['Entry Fees'] + formatted_logs['Exit Fees']
    formatted_logs['Net PnL'] = formatted_logs['PnL'] - formatted_logs['Total Fees']

    final_logs = pd.DataFrame({
        'Entry Date': pd.to_datetime(formatted_logs['Entry Timestamp']).dt.date,
        'Entry Price': formatted_logs['Avg Entry Price'].round(2),
        'Exit Date': pd.to_datetime(formatted_logs['Exit Timestamp']).dt.date,
        'Exit Price': formatted_logs['Avg Exit Price'].round(2),
        'Quantity': formatted_logs['Size'].round(2),
        'Entry Fees ($)': formatted_logs['Entry Fees'].round(2),
        'Exit Fees ($)': formatted_logs['Exit Fees'].round(2),
        'Total Fees ($)': formatted_logs['Total Fees'].round(2),
        'Net PnL ($)': formatted_logs['Net PnL'].round(2),
        'Return (%)': (formatted_logs['Return'] * 100).round(2),
        'Portfolio Value ($)': formatted_logs['Portfolio Value'].round(2)
    })

    if 'Duration' in formatted_logs:
        final_logs['Duration'] = formatted_logs['Duration'].apply(lambda x: f"{x} days")

    return final_logs

In [4]:
# @title Data Download and Preparation
# Step 1: Download AAPL stock data for the last 5 years
print("Step 1: Downloading and cleaning data for AAPL...")
# Calculate start date for last 5 years
from datetime import datetime, timedelta
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=5*365)).strftime('%Y-%m-%d')

df = get_clean_financial_data("AAPL", start_date, end_date)

Step 1: Downloading and cleaning data for AAPL...

Data download complete. Shape: (1256, 5)


In [5]:
# @title Technical Indicator Calculation, Trading Signal Generation and Performance Statistics
# Step 2: Calculate Simple Moving Averages (SMA)
if df is not None:
    print("\nStep 2: Calculating SMA 10 and 20...")
    # Calculate the 10-day Simple Moving Average (SMA)
    df['SMA_10'] = df['Close'].rolling(window=10).mean()
    # Calculate the 20-day Simple Moving Average (SMA)
    df['SMA_20'] = df['Close'].rolling(window=20).mean()

    # Drop any rows with NaN values resulting from the SMA calculation
    df.dropna(inplace=True)
    print("SMA 10 and 20 calculation complete.")

    print("\nStep 3: Creating SMA crossover signals...")
    # Entry when the SMA 10 crosses above the SMA 20
    entries = (df['SMA_10'] > df['SMA_20']) & (df['SMA_10'].shift(1) <= df['SMA_20'].shift(1))
    # Exit when the SMA 10 crosses below the SMA 20
    exits = (df['SMA_10'] < df['SMA_20']) & (df['SMA_10'].shift(1) >= df['SMA_20'].shift(1))
    print("SMA crossover signals created.")

    print("\nStep 4: Backtesting the strategy and generating transaction log...")
    pf = vbt.Portfolio.from_signals(
        close=df['Close'],
        entries=entries,
        exits=exits,
        size=np.inf,
        fees=0.002,
        slippage=0.001,
        freq='D',
        init_cash=10000
    )

    trade_logs = format_trade_logs(pf, pf.trades.records_readable)

    print("\nTransaction Log:")
    if not trade_logs.empty:
        print(trade_logs.to_string())
    else:
        print("No trades were executed based on the SMA crossover strategy.")

    # Display comprehensive backtest performance metrics
    print("\nBacktest Statistics:")
    print(pf.stats())


Step 2: Calculating SMA 10 and 20...
SMA 10 and 20 calculation complete.

Step 3: Creating SMA crossover signals...
SMA crossover signals created.

Step 4: Backtesting the strategy and generating transaction log...

Transaction Log:
    Entry Date  Entry Price   Exit Date  Exit Price  Quantity  Entry Fees ($)  Exit Fees ($)  Total Fees ($)  Net PnL ($)  Return (%)  Portfolio Value ($)
0   2021-01-25       139.40  2021-02-11      131.74     71.59           19.96          18.86           38.82      -626.40       -5.89              9412.42
1   2021-03-23       119.70  2021-03-31      119.08     78.48           18.79          18.69           37.48      -123.56       -0.92              9326.34
2   2021-04-05       122.98  2021-05-05      124.88     75.68           18.62          18.90           37.52        68.69        1.14              9432.55
3   2021-06-14       127.67  2021-08-09      142.88     73.73           18.83          21.07           39.90      1041.12       11.48             